Preprocessing

In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 55.2 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Mediapipe model and utilities
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
# Function to detect and extract landmarks using Mediapipe
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

  # Function to extract keypoints from Mediapipe results
def extract_keypoints(results):
    # Pose landmarks (33 landmarks)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    # Left hand landmarks (21 landmarks)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    # Right hand landmarks (21 landmarks)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)

    # Concatenate pose and hand keypoints
    return np.concatenate([pose, lh, rh])

In [ ]:
# Load metadata from JSON
metadata = {}
with open('/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/WLASL_v0.3.json', 'r') as file:
    metadata = json.load(file)

labelMap = {}
for i in metadata:
    label = i['gloss']
    for instance in i['instances']:
        video_id = int(instance['video_id'])
        frame_start = instance['frame_start']
        frame_end = instance['frame_end']
        fps = instance['fps']
        labelMap[video_id] = [label, frame_start, frame_end, fps]

In [ ]:
# Create directory for saving processed data
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/Models/MediaPipe_NoFace'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Iterate through videos in the dataset folder
video_path = '/content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/DataSet/videos'

In [ ]:
# Function to process a single video
def process_video(video):
    video_id = int(os.path.splitext(video)[0])
    if video_id not in labelMap:
        return

    label, start_frame, end_frame, fps = labelMap[video_id]

    # Open video file
    cap = cv2.VideoCapture(os.path.join(video_path, video))
    cap.set(cv2.CAP_PROP_FPS, fps)

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        keypoints_data = []
        frame_count = 0
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
            frame_count += 1

            # Skip frames outside the valid range
            if frame_count < start_frame or (end_frame != -1 and frame_count > end_frame):
                continue

            # Process frame and extract keypoints
            image, results = mediapipe_detection(image, holistic)
            keypoints = extract_keypoints(results)
            keypoints_data.append(keypoints)

        # Save keypoints for the entire video
        action_path = os.path.join(DATA_PATH, label)
        video_path_save = os.path.join(action_path, f'{video_id}_keypoints.npy')
        np.save(video_path_save, np.array(keypoints_data))

    cap.release()


# Pre-create all necessary directories to avoid concurrency issues
unique_labels = set(labelMap[video_id][0] for video_id in labelMap)
for label in unique_labels:
    action_path = os.path.join(DATA_PATH, label)
    os.makedirs(action_path, exist_ok=True)  # Use exist_ok=True to avoid errors if the directory exists

# List all video files
video_files = [video for video in os.listdir(video_path) if video.endswith('.mp4')]

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(process_video, video_files), total=len(video_files), desc="Processing Videos"))


Processing Videos: 100%|██████████| 11980/11980 [2:50:11<00:00,  1.17it/s]
